## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///mlflow.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1746618121757, experiment_id='0', last_update_time=1746618121757, lifecycle_stage='active', name='Default', tags={}>]

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
import pandas as pd

# Set the experiment name
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    # Load data
    data = load_iris(as_frame=True)  # returns a pandas DataFrame and a Series
    X = data.data                    # DataFrame with proper column names
    y = data.target                  # Series with labels

    # Log model parameters
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    # Train the model
    lr = LogisticRegression(**params).fit(X, y)

    # Predict and log accuracy
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    # Log the model with input example (to generate signature)
    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="models",
        input_example=X.iloc[[0]]  # Helps auto-infer input signature
    )

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


/usr/local/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


default artifacts URI: 'mlflow-artifacts:/1/1e51a17dd8e3448f928c9ee5bafdc5e3/artifacts'
🏃 View run vaunted-gnu-156 at: http://127.0.0.1:5000/#/experiments/1/runs/1e51a17dd8e3448f928c9ee5bafdc5e3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [24]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1746620096656, experiment_id='1', last_update_time=1746620096656, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1746618121757, experiment_id='0', last_update_time=1746618121757, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [20]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [21]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1746620349181, description='', last_updated_timestamp=1746620349203, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1746620349203, current_stage='None', description='', last_updated_timestamp=1746620349203, name='iris-classifier', run_id='ef35021545e94cf5af58834438001112', run_link='', source='mlflow-artifacts:/1/ef35021545e94cf5af58834438001112/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

In [25]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2025/05/07 14:27:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1746620831981, current_stage='None', description='', last_updated_timestamp=1746620831981, name='iris-classifier', run_id='1e51a17dd8e3448f928c9ee5bafdc5e3', run_link='', source='mlflow-artifacts:/1/1e51a17dd8e3448f928c9ee5bafdc5e3/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='2'>